In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
sys.path

['/root/src/kuloko/handler_test',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/src/kuloko/.ipython',
 '../',
 '../',
 '/root/src/geco_commons',
 '/root/src/kuloko/handler',
 '/root/src/geco_commons',
 '/root/src/geco_commons',
 '/root/src/geco_commons']

In [5]:
import sys
sys.path.append("../")

In [7]:
from items.aleister_producer  import AleisterProducer

In [7]:
symbol='BTC'
general_config_mode='DOCKER'
private_api_mode='DEFAULT'

In [8]:
ap = AleisterProducer(symbol, general_config_mode, private_api_mode)

Log path = /root/src/kuloko/log/logging.log
Log path = /root/src/kuloko/log/logging.log


In [10]:
ap.start_realtime_fetch()

Log path = /root/src/kuloko/log/logging.log
Log path = /root/src/kuloko/log/logging.log
Log path = /root/src/kuloko/log/logging.log
Log path = /root/src/kuloko/log/logging.log


KeyboardInterrupt: 

In [7]:
ap.start_record_realtime_data()

Log path = /root/src/kuloko/log/logging.log
Log path = /root/src/kuloko/log/logging.log
Log path = /root/src/kuloko/log/logging.log
Log path = /root/src/kuloko/log/logging.log


/usr/local/lib/python3.7/dist-packages/apscheduler/util.py:95: PytzUsageWarning: The zone attribute is specific to pytz's interface; please migrate to a new time zone provider. For more details on how to do so, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  if obj.zone == 'local':
/usr/local/lib/python3.7/dist-packages/apscheduler/triggers/interval.py:66: PytzUsageWarning: The normalize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  return self.timezone.normalize(next_fire_time)


KeyboardInterrupt: 

In [9]:
data = ap.fetch_hist_data('trade', 'BTC', 20211201,20211211)

In [11]:
# data.

In [ ]:
ap.start_realtime_fetch()

In [ ]:
#!/usr/bin/env python
import pika

connection = pika.BlockingConnection(
    pika.ConnectionParameters(host='localhost'))

channel = connection.channel()

channel.queue_declare(queue='aleister')

def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n - 1) + fib(n - 2)

def on_request(ch, method, props, body):
    self.logger.info(f"[DONE] via MQ. Host={method}, pro:{props}, cd:{ch}")
    n = int(body)

    print(" [.] fib(%s)" % n)
    response = fib(n)

    ch.basic_publish(exchange='',
                     routing_key=props.reply_to,
                     properties=pika.BasicProperties(correlation_id = \
                                                         props.correlation_id),
                     body=str(response))
    ch.basic_ack(delivery_tag=method.delivery_tag)

channel.basic_qos(prefetch_count=1)
channel.basic_consume(queue='aleister', on_message_callback=on_request)

print(" [x] Awaiting RPC requests")
channel.start_consuming()

In [ ]:
import pika,time
class MessageWorker(object):
    def __init__(self):
        self.connection = pika.BlockingConnection(
        pika.ConnectionParameters(host='localhost'))
        self.channel = self.connection.channel()
        self.channel.queue_declare(queue='aleister')

    def on_request(self, ch, method, props, body):
        print('MessageWorker get message ------', body)
        response = self.task(body)
        print('MessageWorker callback on queue ------', props.reply_to)
        ch.basic_publish(exchange='',
                        routing_key=None,
                        properties=pika.BasicProperties(
                            correlation_id=props.correlation_id),
                        body=str(response))
        print('MessageWorker send message to MessageSender')
        ch.basic_ask(delivery_tag=method.delivery_tag)

    def work(self):
        self.channel.basic_qos(prefetch_count=2)
        self.channel.basic_consume(on_message_callback=self.on_request, queue='aleister')
        print('[x]Waiting message...')
        self.channel.start_consuming()


    def task(self,body):
        time.sleep(3)
        return body



worker = MessageWorker()
worker.work()